Code appendix to

# Computational approaches in rigorous sociology: Agent-based computational sociology and computational social science
### Andreas Flache, Michael Mäs & Marijn Keijzer

In Gërxhani, De Graaf & Raub (Eds.) *Handbook of Sociological Science. Contributions to Rigorous Sociology*

---

This notebook contains the code necessarry to replicate the example from the chapter, illustrating the *strength-of-weak-bots* effect.

For a more in-depth analysis of the mechanism, see:

Keijzer & Mäs (2021). The Strength of Weak Bots. *Online Social Networks and Media*, 21(100106). https://doi.org/10.1016/j.osnem.2020.100106

In [1]:
!pip install defSim

import defSim as ds        # for our social influence model simulations
import random              # allows us to set a seed for replicable simulations
import networkx as nx      # handles the construction of networks
import numpy as np

import time
import csv

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for defSim: filename=defSim-0.1.0-cp37-none-any.whl size=64589 sha256=6bda10502ffb60ce1c1b6316297263745449a0ba53d1c030522ba62cadf79be6
  Stored in directory: /root/.cache/pip/wheels/68/3f/17/af48ffb500cf6c9c289021d40986646551032d2f9e21a73f0c
Successfully built defSim


## Functions and procedures

In [2]:
# Function to loop over parameter combinations indefinitely
class Circ(list):
    def __getitem__(self, idx):
        return super(Circ, self).__getitem__(idx % len(self))


# Procedure for a single run of the ABCM
def simulation_run(bot_connectedness=.25,
                   bot_activation_rate=.25,
                   N=100,
                   K=10,
                   max_ticks=250000,
                   homophily=1,
                   seed=None):
    """
    This function
        1. Creates the Axelrod model as a NetworkX graph object with a bot
        2. Executes similarity biased influence between
            a. the BOT and a randomly selected network neighbor 
               (with p = bot_activation_rate)
            b. two randomly selected network neighbors 
               (with p = 1-bot_activation_rate)
        3. Returns graph object when no further influence is possible (or after 
           250,000 ticks)
    """

    ## PHASE 1) INITIALIZATION

    random.seed(seed)   # seed the run for replicability

    # create network object and initialize agent attributes
    G = nx.watts_strogatz_graph(N,K,0.05,seed)
    ds.initialize_attributes(G, realization="random_categorical", 
                             num_features=3, num_traits=3)

    # add the bot to the graph
    agentlist = list(G.nodes())
    G.add_node("bot")
    G.nodes["bot"]["f01"] = 99        # Unique bot trait, used to track 
                                      # the bot's effectiveness
    G.nodes["bot"]["f02"] = 0
    G.nodes["bot"]["f03"] = 0

    friends_of_bot = random.sample(agentlist, int(bot_connectedness * N))
    for j in friends_of_bot:
        G.add_edge("bot", j)

    # calculate the dissimilarity on all edges
    calculator = ds.dissimilarity_calculator.select_calculator("hamming")
    calculator.calculate_dissimilarity_networkwide(G)

    # store the indices of direct and indirect contacts of the bot
    shortest_paths = nx.shortest_path_length(G, 'bot')
    agents_l1 = [key for key, val in shortest_paths.items() if val == 1]
    agents_l2 = [key for key, val in shortest_paths.items() if val >= 2]

    # initialize ticks, stop variable, and tickwise dictionaries for output
    ticks = 0
    stop = False

    ## PHASE 2) SIMULATION

    while (not stop):
        ticks += 1

        # choose sending agent
        if random.random() > bot_activation_rate:
            agent_i = ds.select_focal_agent(G, "random", agentlist=agentlist)
        else:
            agent_i = "bot"

        # choose receiving agent
        agent_j = ["bot"]
        while agent_j == ["bot"]:
            agent_j = [(random.choice([neighbor for neighbor in G[agent_i]]))]

        # exert influence
        ds.influence_sim.spread_influence(G, 
                                          "similarity_adoption", 
                                          agent_i, agent_j, 
                                          "one-to-one", 
                                          calculator, 
                                          homophily=homophily)

        # stop the execution if condition is true
        if (not any([True for i, j, d in G.edges.data('dist') if 0.1 < d < .9]) 
           or ticks > max_ticks):
            return G, ticks


# Procedure for running the experiment (repeated call to simulation_run)
def experiment(id=None):
    N = 100

    connect = [.1,.4]
    activate = [.4]
    proximity = [1]
    homo = [0.25, 1, 4]
    conditions = Circ([(x, y, z, zz) for x in connect 
                                     for y in activate 
                                     for z in proximity 
                                     for zz in homo])

    bot_connectedness, bot_activation_rate, proxweight, homophily = conditions[id]

    seed = random.randint(10000, 99999)
    random.seed(seed)

    G, ticks = simulation_run(bot_connectedness= bot_connectedness,
                              bot_activation_rate=bot_activation_rate,
                              homophily=homophily,
                              seed=seed,
                              max_ticks=200000)

    shortest_paths = nx.shortest_path_length(G, 'bot')
    agents_l1 = [key for key, val in shortest_paths.items() if val == 1]
    agents_l2 = [key for key, val in shortest_paths.items() if val >= 2]

    output_line = dict(
        Seed=seed,
        BotConnectedness=bot_connectedness,
        BotActivationRate=bot_activation_rate,
        N=N,
        Ticks=ticks,
        DiffTotal=len([1 for i in G.nodes() if G.nodes[i]['f01'] == 99]),
        NagentsL1=len(agents_l1),
        DiffAgentsL1=len([1 for i in agents_l1 if G.nodes[i]['f01'] == 99]),
        NagentsL2=len(agents_l2),
        DiffAgentsL2=len([1 for i in agents_l2 if G.nodes[i]['f01'] == 99]),
        ProximityWeight=proxweight,
        Homophily=homophily
    )

    a_file = open("bots_homophily.csv", "a")
    writer = csv.writer(a_file)

    writer.writerow(output_line.values())

    a_file.close()

## Running the experiment

In [ ]:
number_of_conditions = 6       # 2 (connectivity) X 3 (homophily)
number_of_repetitions = 25     # times each condition needs to be run
number_of_runs = number_of_conditions * number_of_repetitions

from tqdm import tqdm
for id in tqdm(range(number_of_runs)):
    experiment(id)

100%|██████████| 150/150 [33:00<00:00, 13.20s/it]
